In [46]:
from qiskit import transpile
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from scipy.sparse import diags

from linear_solvers import NumPyLinearSolver, HHL
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.quantum_info import Statevector
import sys
import os
os.environ["Q_ALCHEMY_API_KEY"] = "JnvkpMCsyr4nB9nHcwa6CbxqhtZXyF1b"
sys.path.append('..')
from q_alchemy.qiskit import QAlchemyInitialize
from qiskit.opflow import (
    Z,
    I,
    StateFn,
    TensoredOp,
    ExpectationBase,
    CircuitSampler,
    ListOp,
    ExpectationFactory,
    ComposedOp,
)


In [47]:
def calculate_norm(qc: QuantumCircuit, nb: int, nl: int, na:int) -> float:
        """Calculates the value of the euclidean norm of the solution.

        Args:
            qc: The quantum circuit preparing the solution x to the system.

        Returns:
            The value of the euclidean norm of the solution.
        """

        # Create the Operators Zero and One
        zero_op = (I + Z) / 2
        one_op = (I - Z) / 2

        # Norm observable
        observable = one_op ^ TensoredOp((nl + na) * [zero_op]) ^ (I ^ nb)
        norm_2 = (~StateFn(observable) @ StateFn(qc)).eval()
        

        return np.real(np.sqrt(norm_2))

In [48]:
def get_solution_vector(solution, n):
    solution_vector = Statevector(solution.state).data[n:n+8].real
    norm = solution.euclidean_norm
    return norm * solution_vector / np.linalg.norm(solution_vector)

Define Matrices and solutions

In [49]:
A = np.array([[1.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [2.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [3.0, 4.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 5.0, 6.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 10.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 12.0]])

# Ensure Hermitian property (conjugate transpose)
A = (A + A.T) / 2.0
#b = [1, 0, 0, 0, 0, 0, 0, 0]
b = np.random.rand(8)
b = b/np.linalg.norm(b) #normalize solution

Initialize the solution state using Q Alchemy

In [50]:
sp_org = QAlchemyInitialize(b, opt_params={'max_fidelity_loss':0.0})
sp_org.definition.draw(fold=-1)

┌──────────────────────────────────────────────────────────────────────────┐
q_0: ┤0                                                                         ├
     │                                                                          │
q_1: ┤1 LRSP(0.13465,0.061692,0.21235,0.66501,0.35048,0.033982,0.21654,0.56555) ├
     │                                                                          │
q_2: ┤2                                                                         ├
     └──────────────────────────────────────────────────────────────────────────┘

In [51]:
qc = transpile(sp_org.definition, basis_gates=["id", "rx", "ry", "rz", "cx"])
num_q1 = qc.num_qubits
print('state prepared by Q-alchemy',Statevector(qc).data.real)
print('Target state',b)
qc.draw()

state prepared by Q-alchemy [0.04628963 0.10709471 0.29219531 0.67601684 0.07160833 0.1656715
 0.25449191 0.58878706]
Target state [0.13465084 0.06169202 0.21235015 0.6650149  0.35047703 0.03398187
 0.21653805 0.56555123]


global phase: 3π/2
      ┌────────────┐                              
q_0: ─┤ Ry(2.3256) ├──────────────────────────────
     ┌┴────────────┤     ┌─────────────┐┌────────┐
q_1: ┤ Ry(0.11643) ├──■──┤ Ry(-2.7234) ├┤ Rz(-π) ├
     └─────────────┘┌─┴─┐└┬────────────┤└────────┘
q_2: ───────────────┤ X ├─┤ Ry(1.4578) ├──────────
                    └───┘ └────────────┘

In [54]:
qal_hhl = HHL().solve(A, qc)

num_q = qal_hhl.state.num_qubits
qal_hhl.state.draw()

┌──────────────────┐┌───────┐        ┌──────────┐
q123045_0: ┤0                 ├┤9      ├────────┤9         ├
           │                  ││       │        │          │
q123045_1: ┤1 circuit-1153141 ├┤10     ├────────┤10        ├
           │                  ││       │        │          │
q123045_2: ┤2                 ├┤11     ├────────┤11        ├
           └──────────────────┘│       │┌──────┐│          │
q123046_0: ────────────────────┤0      ├┤8     ├┤0         ├
                               │       ││      ││          │
q123046_1: ────────────────────┤1      ├┤7     ├┤1         ├
                               │       ││      ││          │
q123046_2: ────────────────────┤2      ├┤6     ├┤2         ├
                               │   QPE ││      ││   QPE_dg │
q123046_3: ────────────────────┤3      ├┤5     ├┤3         ├
                               │       ││      ││          │
q123046_4: ────────────────────┤4      ├┤4     ├┤4         ├
                               │       ││  1/x ││          │
q123046_5: ────────────────────┤5      ├┤3     ├┤5         ├
                               │       ││      ││          │
q123046_6: ────────────────────┤6      ├┤2     ├┤6         ├
                               │       ││      ││          │
q123046_7: ────────────────────┤7      ├┤1     ├┤7         ├
                               │       ││      ││          │
q123046_8: ────────────────────┤8      ├┤0     ├┤8         ├
                               └───────┘│      │└──────────┘
  q123047: ─────────────────────────────┤9     ├────────────
                                        └──────┘

In [56]:
classical_solution = NumPyLinearSolver().solve(A, b)
hhl_default = HHL().solve(A, b) #using IBM's default state preparation method

x0 = 2**(num_q-1)

qal_sol = get_solution_vector(qal_hhl, x0)
default_sol = get_solution_vector(hhl_default, x0)
classic_sol = classical_solution.state

print('solution with Q-alchemy:', qal_sol)
print('solution with IBM:', default_sol)
print('classical solution:', classic_sol)



solution with Q-alchemy: [ 0.16023144 -0.1844711   0.08525284  0.04139365 -0.02533618  0.03109089
  0.01572156  0.0087781 ]
solution with IBM: [-0.2167443   0.07956532  0.06442473  0.05697068  0.0131625   0.03245351
  0.06177198 -0.0365057 ]
classical solution: [-0.21658872  0.07965013  0.06397977  0.05751934  0.01306929  0.032374
  0.06146704 -0.03619385]


In [ ]:
print('Q-alchemy euclidean norm:', qal_hhl.euclidean_norm)
print('IBM euclidean norm:', hhl_default.euclidean_norm)
print('Classical euclidean norm:', classical_solution.euclidean_norm)


Q-alchemy euclidean norm: 0.15417175794858626
IBM euclidean norm: 0.20401954342097062
Classical euclidean norm: 0.20335947032118262


Check the state prepared by Q alchemy and IBM

In [57]:
hhl_default.state.data.pop(-1)
state_IBM = Statevector(hhl_default.state).data.real
hhl_default.state.draw()



┌──────────────────┐┌───────┐        
q136358_0: ┤0                 ├┤9      ├────────
           │                  ││       │        
q136358_1: ┤1 circuit-1307712 ├┤10     ├────────
           │                  ││       │        
q136358_2: ┤2                 ├┤11     ├────────
           └──────────────────┘│       │┌──────┐
q136359_0: ────────────────────┤0      ├┤8     ├
                               │       ││      │
q136359_1: ────────────────────┤1      ├┤7     ├
                               │       ││      │
q136359_2: ────────────────────┤2      ├┤6     ├
                               │   QPE ││      │
q136359_3: ────────────────────┤3      ├┤5     ├
                               │       ││      │
q136359_4: ────────────────────┤4      ├┤4     ├
                               │       ││  1/x │
q136359_5: ────────────────────┤5      ├┤3     ├
                               │       ││      │
q136359_6: ────────────────────┤6      ├┤2     ├
                               │       ││      │
q136359_7: ────────────────────┤7      ├┤1     ├
                               │       ││      │
q136359_8: ────────────────────┤8      ├┤0     ├
                               └───────┘│      │
  q136360: ─────────────────────────────┤9     ├
                                        └──────┘

In [58]:
hhl_default.state.data.pop(-1)
state_IBM = Statevector(hhl_default.state).data.real
hhl_default.state.draw()


┌──────────────────┐┌───────┐
q136358_0: ┤0                 ├┤9      ├
           │                  ││       │
q136358_1: ┤1 circuit-1307712 ├┤10     ├
           │                  ││       │
q136358_2: ┤2                 ├┤11     ├
           └──────────────────┘│       │
q136359_0: ────────────────────┤0      ├
                               │       │
q136359_1: ────────────────────┤1      ├
                               │       │
q136359_2: ────────────────────┤2      ├
                               │   QPE │
q136359_3: ────────────────────┤3      ├
                               │       │
q136359_4: ────────────────────┤4      ├
                               │       │
q136359_5: ────────────────────┤5      ├
                               │       │
q136359_6: ────────────────────┤6      ├
                               │       │
q136359_7: ────────────────────┤7      ├
                               │       │
q136359_8: ────────────────────┤8      ├
                               └───────┘
  q136360: ─────────────────────────────

In [59]:
hhl_default.state.data.pop(-1)
state_IBM = Statevector(hhl_default.state).data.real
hhl_default.state.draw()

┌──────────────────┐
q136358_0: ┤0                 ├
           │                  │
q136358_1: ┤1 circuit-1307712 ├
           │                  │
q136358_2: ┤2                 ├
           └──────────────────┘
q136359_0: ────────────────────
                               
q136359_1: ────────────────────
                               
q136359_2: ────────────────────
                               
q136359_3: ────────────────────
                               
q136359_4: ────────────────────
                               
q136359_5: ────────────────────
                               
q136359_6: ────────────────────
                               
q136359_7: ────────────────────
                               
q136359_8: ────────────────────
                               
  q136360: ────────────────────

In [63]:
qal_hhl.state.data.pop(-1)
qal_hhl.state.data.pop(-1)
qal_hhl.state.draw()



┌──────────────────┐┌───────┐
q123045_0: ┤0                 ├┤9      ├
           │                  ││       │
q123045_1: ┤1 circuit-1153141 ├┤10     ├
           │                  ││       │
q123045_2: ┤2                 ├┤11     ├
           └──────────────────┘│       │
q123046_0: ────────────────────┤0      ├
                               │       │
q123046_1: ────────────────────┤1      ├
                               │       │
q123046_2: ────────────────────┤2      ├
                               │   QPE │
q123046_3: ────────────────────┤3      ├
                               │       │
q123046_4: ────────────────────┤4      ├
                               │       │
q123046_5: ────────────────────┤5      ├
                               │       │
q123046_6: ────────────────────┤6      ├
                               │       │
q123046_7: ────────────────────┤7      ├
                               │       │
q123046_8: ────────────────────┤8      ├
                               └───────┘
  q123047: ─────────────────────────────

In [64]:
qal_hhl.state.data.pop(-1)
qal_hhl.state.draw()


┌──────────────────┐
q123045_0: ┤0                 ├
           │                  │
q123045_1: ┤1 circuit-1153141 ├
           │                  │
q123045_2: ┤2                 ├
           └──────────────────┘
q123046_0: ────────────────────
                               
q123046_1: ────────────────────
                               
q123046_2: ────────────────────
                               
q123046_3: ────────────────────
                               
q123046_4: ────────────────────
                               
q123046_5: ────────────────────
                               
q123046_6: ────────────────────
                               
q123046_7: ────────────────────
                               
q123046_8: ────────────────────
                               
  q123047: ────────────────────

In [65]:
state_qal = Statevector(qal_hhl.state).data.real


[0.04628963 0.10709471 0.29219531 0.67601684 0.07160833 0.1656715
 0.25449191 0.58878706]


In [67]:
print('state prepared by q_alchemy',state_qal[0:8])
print('state prepared by IBMs default method' ,state_IBM[0:8])
print('Target state',b)

state prepared by q_alchemy [0.04628963 0.10709471 0.29219531 0.67601684 0.07160833 0.1656715
 0.25449191 0.58878706]
state prepared by IBMs default method [0.13465084 0.06169202 0.21235015 0.6650149  0.35047703 0.03398187
 0.21653805 0.56555123]
Target state [0.13465084 0.06169202 0.21235015 0.6650149  0.35047703 0.03398187
 0.21653805 0.56555123]
